In [7]:
import pandas as pd

In [8]:
import numpy as np

In [9]:
import os; import re

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
from sklearn.cross_validation import KFold

In [12]:
from sklearn.grid_search import GridSearchCV

In [13]:
import time
import datetime

In [14]:
from sklearn.linear_model import LogisticRegressionCV

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
features = pd.read_csv('./_features/features.csv', index_col='match_id')

In [35]:
dict_names = os.listdir("./_data/data/dictionaries")

In [4]:
Xtest = pd.read_csv('./_features_test/features_test.csv', index_col='match_id')

###### create dict of dicts with object description

In [36]:
dicts = dict()
for i,d in enumerate(dict_names):
    dicts[re.sub(r'(.*)\.csv',r'\1',d)] = pd.read_csv('./_data/data/dictionaries/%s' % d)
#print dicts

In [42]:
X_na[keys]

,r1_hero,r2_hero,r3_hero,r4_hero,r5_hero,d1_hero,d2_hero,d3_hero,d4_hero,d5_hero,lobby_type,start_time,first_blood_player1,first_blood_player2
match_id,,,,,,,,,,,,,,
0,11,67,29,20,105,4,42,21,37,84,7,1430198770,9,0
1,42,49,67,37,26,39,88,79,7,12,0,1430220345,7,0
2,33,98,20,27,4,22,66,86,29,80,7,1430227081,3,0
3,29,30,75,37,41,96,48,15,102,20,1,1430263531,0,0
4,13,27,30,72,93,26,69,22,25,8,7,1430282290,6,0
5,11,20,28,25,65,55,52,3,73,48,1,1430284186,7,0
8,8,57,7,21,36,22,30,72,101,47,1,1430293701,7,0
9,35,15,83,29,101,100,25,26,44,96,7,1430299335,0,0
11,17,91,53,72,30,90,96,35,19,7,1,1430308974,5,0


###### drop final attributes

In [17]:
X = features.drop(features.columns[-6:],1)

###### found skips

In [18]:
counts=X.count()

In [19]:
skips=counts[counts.values < 97230].keys()

###### не в каждом матче FB происходит в первые 5 минут, не в каждом матче летучий курьер приобретается одной из команд в первые 5 минут

In [20]:
skips[[0,-2]]

Index([u'first_blood_time', u'dire_flying_courier_time'], dtype='object')

###### replace na items to ZERO

In [21]:
X_na = X.fillna(0)

###### target var - radiant_win

In [22]:
y=features.radiant_win

###### create cross validator

In [23]:
cv = KFold(y.size,shuffle=True, n_folds=5)

In [20]:
sc=StandardScaler()

Xs=sc.fit_transform(X_na)

In [27]:
keys=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

keys.append('lobby_type')

Xdr = X_na.drop(keys,1)

In [28]:
Xdr2=Xdr.drop(X.keys()[pd.Series(X.keys()).str.contains('first_blood_player', na=False)],1)

In [24]:
keys=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])
#keys.append('lobby_type')
keys=keys+['lobby_type','start_time']+X.keys()[pd.Series(X.keys()).str.contains('first_blood_player', na=False)].tolist()

Xdr2 = X_na.drop(keys,1)

In [25]:
scdr=StandardScaler()

Xsdr=scdr.fit_transform(Xdr2)

In [48]:
est1 = LogisticRegressionCV(Cs=20,cv=cv,scoring='roc_auc',n_jobs=-1)

In [49]:
start_time = datetime.datetime.now()
est1.fit(Xsdr,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:00:17.918000


In [50]:
est1.scores_[1].max()

0.72075742226010331

In [38]:
est1.scores_[1].max()

0.72075651602744528

In [26]:
ky=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

frames = []
for i in ky:
    frames.append(pd.DataFrame(X[i]).rename(index=str, columns={i: "hero"}))

all_hero = pd.concat(frames,ignore_index=True)
N=all_hero['hero'].unique().size

In [26]:
N=112

In [27]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [28]:
XX=np.column_stack((Xsdr,X_pick))

In [38]:
X_na

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16
5,1430284186,1,11,5,1961,1461,19,0,1,6,...,1,-27,2,-86,212,4,4,4,0,-43
8,1430293701,1,8,3,967,1136,7,1,0,8,...,2,-35,6,-86,182,3,6,3,0,10
9,1430299335,7,35,5,2117,1252,16,0,0,6,...,1,26,119,-61,0,2,3,4,0,-15
11,1430308974,1,17,5,1527,906,10,0,1,7,...,1,9,0,-84,226,1,3,4,0,26


In [43]:
estD.C_

array([ 0.07196857])

In [226]:
list(range(10,21))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [29]:
estD = LogisticRegressionCV(Cs=15,cv=cv,scoring='roc_auc',n_jobs=-1)

In [30]:
start_time = datetime.datetime.now()
estD.fit(XX,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:01:23.792000


In [35]:
estD.scores_[1].max()

0.75633555643778616

In [44]:
estD.Cs_

array([  1.00000000e-04,   3.72759372e-04,   1.38949549e-03,
         5.17947468e-03,   1.93069773e-02,   7.19685673e-02,
         2.68269580e-01,   1.00000000e+00,   3.72759372e+00,
         1.38949549e+01,   5.17947468e+01,   1.93069773e+02,
         7.19685673e+02,   2.68269580e+03,   1.00000000e+04])

In [24]:
0.75845107346698071

0.7584510734669807

In [32]:
scores = []

In [33]:
for i in estD.scores_[1]:
    scores.append(i.mean())
    print i.mean()
scores.max()

0.746024410496
0.744660676263
0.750138479868
0.750640021042
0.752097933754


AttributeError: 'list' object has no attribute 'max'

In [46]:
XX.shape

(97230L, 201L)

In [28]:
est = LogisticRegressionCV(Cs=15,cv=cv,scoring='roc_auc',n_jobs=-1)

In [29]:
start_time = datetime.datetime.now()
est.fit(Xs,y)
print 'Time elapsed:', datetime.datetime.now() - start_time    

Time elapsed: 0:00:19.256000


In [33]:
est.scores_[1].max()

0.7206460881333201

In [28]:
X_pick.shape

(97230L, 112L)

In [95]:
pest = PCA(n_components=180)
pest.fit(XX.T)
#est.components_
disp=pest.explained_variance_ratio_
disp.sort()
rd=disp[::-1]
rd.cumsum()

array([ 0.04079027,  0.08074931,  0.1203152 ,  0.15971658,  0.19880959,
        0.2378034 ,  0.27649712,  0.31503064,  0.35344849,  0.39162472,
        0.41593185,  0.42990261,  0.44353182,  0.45640762,  0.46917486,
        0.48188077,  0.49447547,  0.5070586 ,  0.51959347,  0.53208532,
        0.54451215,  0.55681476,  0.56751028,  0.57796498,  0.5880962 ,
        0.59765106,  0.60718088,  0.61664377,  0.62608944,  0.63549868,
        0.64485528,  0.65419884,  0.66352509,  0.67281533,  0.68201611,
        0.69115436,  0.70009723,  0.70854641,  0.7168691 ,  0.72491695,
        0.73283184,  0.74062176,  0.74826358,  0.75590125,  0.7635132 ,
        0.77109004,  0.77860634,  0.78608603,  0.79355206,  0.80099485,
        0.80841933,  0.81523581,  0.82165025,  0.82801829,  0.8332951 ,
        0.83829627,  0.84322965,  0.8481287 ,  0.85298991,  0.85772344,
        0.86243577,  0.86710539,  0.87177407,  0.87641204,  0.88101395,
        0.88558019,  0.88974527,  0.89368719,  0.89738322,  0.90

In [96]:
pcX=pest.components_

In [75]:
pcX.T.shape

(97230L, 70L)

In [84]:
XX.shape

(97230L, 201L)

In [127]:
keys

['lobby_type',
 'r1_hero',
 'r2_hero',
 'r3_hero',
 'r4_hero',
 'r5_hero',
 'd1_hero',
 'd2_hero',
 'd3_hero',
 'd4_hero',
 'd5_hero']

In [190]:
#params_grid={"n_estimators":[10,20,30]}

In [191]:
#gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")

In [193]:
times = pd.DataFrame(columns=['n_estimators','time','score'],index=[10,20,30,60])

In [194]:
for n in [10,20,30,60]:
    params_grid={"n_estimators":[n]}
    gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")
    start_time = datetime.datetime.now()
    #times['n_estimators:' + gs.param_grid.values()[0][0]] = 
    gs.fit(X_na,y)
    n_est=gs.param_grid.values()[0][0]
    et=(datetime.datetime.now() - start_time).seconds
    sc=gs.best_score_
    times.loc[n_est] = [n_est,et,sc]

#print 'Time elapsed:', datetime.datetime.now() - start_time
times

,n_estimators,time,score
10,10,63,0.66469
20,20,126,0.682274
30,30,184,0.689361
60,60,505,0.700428


In [19]:
op_times = pd.DataFrame(columns=['n_estimators','time','score'],index=[10,20,30,60])

In [35]:
for n in [10]:
    params_grid={"n_estimators":[n],'loss':['deviance'],'subsample':[0.2]}
    gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")
    start_time = datetime.datetime.now()
    #times['n_estimators:' + gs.param_grid.values()[0][0]] = 
    gs.fit(X_na,y)
    n_est=gs.param_grid.values()[0][0]
    et=(datetime.datetime.now() - start_time).seconds
    sc=gs.best_score_
    
    op_times.loc[n_est] = [n_est,et,sc]

#print 'Time elapsed:', datetime.datetime.now() - start_time
op_times

,n_estimators,time,score
10,10,46,0.668749
20,NaN,NaN,NaN
30,30,122,0.684247
60,NaN,NaN,NaN


In [34]:
op_times

,n_estimators,time,score
10,10,44,0.670543
20,NaN,NaN,NaN
30,30,122,0.684247
60,NaN,NaN,NaN


In [23]:
op_times

,n_estimators,time,score
10,NaN,NaN,NaN
20,NaN,NaN,NaN
30,30,195,0.690125
60,NaN,NaN,NaN
